In [1]:
import json
import re
import os
import gzip
import numbers
import numpy as np
import pandas as pd
import platform
import multiprocessing as mp
from functools import partial
import statsmodels.api as sm
import seaborn as sns
import matplotlib as mpl
from IPython.display import display, HTML
from datetime import datetime as dt

spectra_map = {0: 'M', 1: 'K', 2: 'G', 3: 'F', 4: 'A', 5: 'B', 6: 'O', 7: 'X'}
singularity_flags = {
    'TidalLocked': 1,
    'TidalLocked2': 2,
    'TidalLocked4': 4,
    'LaySide': 8,
    'ClockwiseRotate': 16,
    'MultipleSatellites': 32,
}
star_type_map = {
    0: 'Main Sequence',
    1: 'Giant',
    2: 'White Dwarf',
    3: 'Neutron Star',
    4: 'Black Hole',
}
vein_type_map = {
    'Iron': 'Iron', 'Copper': 'Copper', 'Silicium': 'Silicon', 'Titanium': 'Titanium', 
    'Stone': 'Stone', 'Coal': 'Coal', 'Oil' : 'Oil', 'Fireice': 'Fire ice', 'Diamond': 'Kimberlite',
    'Fractal': 'Fractal Silicon', 'Crysrub': 'Organic Crystal', 'Grat': 'Optical Grating', 
    'Bamboo': 'Spiniform Stalagmite Crystal', 'Mag': 'Unipolar Magnets',
}
vein_types = list(vein_type_map.values())
ocean_types = ['Water', 'Sulfuric acid']
base_types = ['Iron', 'Copper', 'Silicon', 'Titanium', 'Stone', 'Coal']
gas_types = ['Hydrogen gas', 'Deuterium gas', 'Fire ice gas',]
S_scaled = gas_types + ['Oil']
M_scaled = [x for x in vein_types if x not in S_scaled]

previous_runs = [
    84459701, 16015752, 22778246, 26027001, 57786559, 51279269, 94520636, 
    32680784, 88834638, 22161804, 87759803, 76768927, 18238299, 43378616,
]

%load_ext autotime

time: 178 µs (started: 2021-10-13 12:07:12 +01:00)


In [2]:
export_path = {
    # original Windows location
    'Chonyi': '/mnt/c/' + r'\Users\Drake\Documents\Dyson Sphere Program\dsp_seedexporter'.replace('\\', '/'),
    
    # NTFS mount in Linux
    'chonyi': '/media/drake/Windows/Users/Drake/Documents/Dyson Sphere Program/dsp_seedexporter/',
    
    # copied from Chonyi
    'blincoln': os.path.expanduser('~/data/dsp/seeds'),
}[platform.node()]
seed_re = re.compile(r'seed_(?P<seed>\d{8}).json.gz')

time: 577 µs (started: 2021-10-13 12:07:12 +01:00)


In [3]:
files = sorted(filter(seed_re.match, os.listdir(export_path)))
seeds = list(map(lambda x: seed_re.match(x).group('seed'), files))
print('{:d} files detected'.format(len(files)))

6882 files detected
time: 32 ms (started: 2021-10-13 12:07:12 +01:00)


In [4]:
def process_universe(func, filename):
    file = os.path.join(export_path, filename)
    try:
        with gzip.GzipFile(file, mode='r') as fd:
            universe = json.load(fd)
            universe['meta']['mtime'] = dt.fromtimestamp(os.stat(file).st_mtime)
    except EOFError as e:
        raise ValueError('Bad file: ' + file)
    return func(universe)

def passthrough(x):
    return x

subset = ['seed_18238299.json.gz', 'seed_94520636.json.gz', 'seed_94520637.json.gz']
with mp.Pool(mp.cpu_count() - 1) as pool:
    universes = pool.map(partial(process_universe, passthrough), subset, chunksize=100)
universe = universes[0]

time: 73.1 ms (started: 2021-10-13 12:07:12 +01:00)


In [5]:
def distance(star):
    return sum(np.fromiter(star['position'].values(), dtype='float') ** 2) ** 0.5

distance_df = pd.DataFrame.from_records(
    data=[(star_name, distance(star)) for star_name, star in universe['star'].items()], 
    columns=['star_name', 'abs. dist.']
).sort_values('abs. dist.')
distance_df

,star_name,abs. dist.
0,Procyon,0.000000
1,Tramontana,2.960353
16,Zeta Arietis,4.005012
17,27 Ophiuchii,4.611032
4,Lambda Sagittae,5.189216
...,...,...
40,ζ Chamaeleontis,27.360977
45,Cymbae,30.245616
51,Theta Puppis,33.166251
60,Arided,36.442384


time: 10.7 ms (started: 2021-10-13 12:07:12 +01:00)


In [6]:
home_star = distance_df.iloc[0]
assert home_star['abs. dist.'] == 0.
home_system = universe['star'][home_star['star_name']]

time: 1.76 ms (started: 2021-10-13 12:07:12 +01:00)


In [7]:
def map_strings(universe):
    planet_types = []
    star_types = []
    singularities = []
    planet_count = 0
    for star in universe['star'].values():
        try:
            star_types.append((star['type'], star['typeString']))
        except KeyError:
            pass
        for planet in star['planet'].values():
            try:
                singularities.append((planet['singularity'], planet['singularityString']))
            except KeyError:
                pass
            try:
                planet_types.append((planet['type'], planet['typeString']))
            except KeyError:
                pass
            planet_count += 1
    return (
        np.unique(planet_types, axis=0), 
        np.unique(singularities, axis=0), 
        np.unique(star_types, axis=0), 
        planet_count
    )


with mp.Pool(mp.cpu_count() - 1) as pool:
    planet_types, singularities, star_types, planet_count = zip(*pool.imap_unordered(partial(process_universe, map_strings), files, chunksize=100))
planet_types = np.unique(np.concatenate([x for x in planet_types if len(x) > 0]), axis=0)
singularities = np.unique(np.concatenate([x for x in singularities if len(x) > 0]), axis=0)
star_types = np.unique(np.concatenate([x for x in star_types if len(x) > 0]), axis=0)


time: 9.27 s (started: 2021-10-13 12:07:12 +01:00)


In [8]:
display(pd.DataFrame.from_records(planet_types, columns=['code', 'type']))
display(pd.DataFrame.from_records(star_types, columns=['code', 'type']))

,code,type
0,1,Lava
1,1,Volcanic Ash
2,2,Mediterranean
3,2,Oceanic Jungle
4,2,Prairie
5,2,Red Stone
6,2,Sakura Ocean
7,2,Waterworld
8,3,Arid Desert
9,3,Ashen Gelisol


,code,type
0,0,A type star
1,0,B type star
2,0,F type star
3,0,G type star
4,0,K type star
5,0,M type star
6,0,O type star
7,1,Blue giant
8,1,Red giant
9,1,White giant


time: 10 ms (started: 2021-10-13 12:07:21 +01:00)


In [9]:
OIL_SPEED_MULTIPLIER = 4e-5
DYSON_SPHERE_BUFFER = 1.35

def reorder(df):
    resource_cols = vein_types + gas_types
    first_cols = [x for x in df.columns if x not in resource_cols]
    df[resource_cols] = df[resource_cols].fillna(0)
    return df[first_cols + resource_cols]

def scan_systems(universe):
    home_star_id = universe['meta']['birthStarId']
    home_planet_id = universe['meta']['birthPlanetId']
    seed = universe['meta']['seed']
    mtime = universe['meta']['mtime']
    stars = []
    planets = []
    
    for star in universe['star'].values():
        max_dyson_rad = star['dysonRadius'] * 2

        star_dict = {
            'seed': seed,
            'star': star['name'],
            'home_system': star['id'] == home_star_id,
            'spectrum': spectra_map[star['spectr']],
            'type': star_type_map[star['type']],
            'luminosity': star['luminosity'],
        }
        stars.append(star_dict)

        for planet in star['planet'].values():
            planet_dict = {
                'seed': seed,
                'star': star['name'],
                'planet': planet['name'],
                'home_system': star['id'] == home_star_id,
                'homeworld': planet['id'] == home_planet_id,
                'type': planet.get('typeString', None),
                'tidal_lock': (planet['singularity'] & singularity_flags['TidalLocked']) > 0,
                'in_sphere': planet['sunDistance'] <= max_dyson_rad / DYSON_SPHERE_BUFFER,
                'ocean': planet.get('waterItem', None),
                
            }
            if 'vein' in planet:
                planet_veins = {vein_type_map[k]: v for k, v in planet['vein'].items()}
            elif 'gas' in planet:
                planet_veins = {(k + ' gas'): float(v['gasSpeed']) for k,v in planet['gas'].items()}
            else:
                raise ValueError('Bad planet: {}'.format(planet['name']))
            assert all(isinstance(x, (int, float)) for x in planet_veins.values()), planet_veins
            planets.append({**planet_dict, **planet_veins})
            
    planet_df = pd.DataFrame.from_records(planets)
    planet_df['Oil'] *= OIL_SPEED_MULTIPLIER
    
    star_df = pd.DataFrame.from_records(stars)
    systems = planet_df.groupby('star')
    star_resources = systems.sum().reindex(vein_types + gas_types, axis='columns')
    star_resources['planets'] = systems['planet'].count()
    oceans = (systems['ocean'].value_counts().unstack() > 0)[['Water', 'Sulfuric acid']]
    oceans.reindex(star_df['star'], fill_value=False)
    star_df = (star_df
               .merge(star_resources, left_on='star', right_index=True, validate='1:1')
               .merge(oceans.reindex(star_df['star'], fill_value=False), 
                      left_on='star', right_index=True, validate='1:1'))
    
    seed_df =  pd.DataFrame.from_records([{'seed': seed, 'ts': mtime}])
    
    return star_df, planet_df, seed_df

star_df, planet_df, seed_df = scan_systems(universe)
display(star_df.head())
display(planet_df.head())
display(seed_df.head())

,seed,star,home_system,spectrum,type,luminosity,Iron,Copper,Silicon,Titanium,...,Organic Crystal,Optical Grating,Spiniform Stalagmite Crystal,Unipolar Magnets,Hydrogen gas,Deuterium gas,Fire ice gas,planets,Water,Sulfuric acid
0,18238299,Procyon,True,G,Main Sequence,0.978408,23539812.0,33240354.0,13467962.0,13893383.0,...,0.0,0.0,0.0,0.0,0.250872,0.0,0.597154,4,True,False
1,18238299,Tramontana,False,M,Main Sequence,0.583942,14053173.0,28310788.0,3529581.0,37695658.0,...,0.0,2664609.0,0.0,0.0,0.000000,0.0,0.000000,3,True,False
2,18238299,Gamma Vulpeculae,False,K,Main Sequence,0.823254,36914259.0,41160402.0,20979566.0,37358159.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,3,True,False
3,18238299,YedPrior,False,M,Main Sequence,0.497194,26414756.0,18726224.0,19573996.0,39492924.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,3,True,False
4,18238299,Lambda Sagittae,False,K,Main Sequence,0.817663,9531719.0,1975948.0,4960219.0,31078570.0,...,0.0,958814.0,0.0,0.0,0.000000,0.0,0.000000,2,True,False


,seed,star,planet,home_system,homeworld,type,tidal_lock,in_sphere,ocean,Iron,...,Hydrogen gas,Oil,Fire ice,Fractal Silicon,Optical Grating,Spiniform Stalagmite Crystal,Kimberlite,Organic Crystal,Deuterium gas,Unipolar Magnets
0,18238299,Procyon,Procyon I,True,False,Lava,False,True,None,19746067.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18238299,Procyon,Procyon II,True,False,Gobi,False,False,None,892945.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18238299,Procyon,Procyon III,True,False,Ice Giant,False,False,None,NaN,...,0.250872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18238299,Procyon,Procyon IV,True,True,Mediterranean,False,False,Water,2900800.0,...,NaN,53.22388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18238299,Tramontana,Tramontana I,False,False,Arid Desert,False,True,None,1682756.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,seed,ts
0,18238299,2021-09-07 18:25:41.524348


time: 38.7 ms (started: 2021-10-13 12:07:21 +01:00)


In [10]:
with mp.Pool(mp.cpu_count() - 1) as pool:
    star_dfs, planet_dfs, seed_dfs = zip(*pool.imap_unordered(partial(process_universe, scan_systems), files, chunksize=100))
star_df = reorder(pd.concat(star_dfs))
planet_df = reorder(pd.concat(planet_dfs))
seed_df = pd.concat(seed_dfs)

time: 25.6 s (started: 2021-10-13 12:07:21 +01:00)


In [11]:
home_systems = star_df.query('home_system == True').copy()
home_systems.head()

,seed,star,home_system,spectrum,type,luminosity,planets,Water,Sulfuric acid,Iron,...,Fire ice,Kimberlite,Fractal Silicon,Organic Crystal,Optical Grating,Spiniform Stalagmite Crystal,Unipolar Magnets,Hydrogen gas,Deuterium gas,Fire ice gas
0,11469817,Caph,True,G,Main Sequence,1.054362,4,True,False,24162823.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.838646,0.033663,0.000000
0,11479026,Beta Reticuli,True,G,Main Sequence,0.983889,4,True,False,24764229.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.846519,0.034590,0.000000
0,11479484,Avior,True,G,Main Sequence,0.941180,4,True,False,23442756.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.899259,0.036709,0.000000
0,11481106,58 Cancri,True,G,Main Sequence,1.002826,4,True,False,6534764.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.277222,0.000000,0.572811
0,11490763,Bunda,True,G,Main Sequence,0.966533,4,True,False,23625432.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.750012,0.031936,0.000000


time: 22.6 ms (started: 2021-10-13 12:07:47 +01:00)


In [12]:
home_system_planets = planet_df.query('home_system == True').copy()
home_system_planets.head()

,seed,star,planet,home_system,homeworld,type,tidal_lock,in_sphere,ocean,Iron,...,Fire ice,Kimberlite,Fractal Silicon,Organic Crystal,Optical Grating,Spiniform Stalagmite Crystal,Unipolar Magnets,Hydrogen gas,Deuterium gas,Fire ice gas
0,11469817,Caph,Caph I,True,False,Lava,False,True,None,21705365.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
1,11469817,Caph,Caph II,True,False,Gobi,False,False,None,530001.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
2,11469817,Caph,Caph III,True,False,Gas Giant,False,False,None,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.838646,0.033663,0.0
3,11469817,Caph,Caph IV,True,True,Mediterranean,False,False,Water,1927457.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
0,11479026,Beta Reticuli,Beta Reticuli I,True,False,None,False,True,None,20282234.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0


time: 22.5 ms (started: 2021-10-13 12:07:47 +01:00)


In [13]:
home_system_planets['TL|IS'] = home_system_planets['tidal_lock'] | home_system_planets['in_sphere']
home_system_planets['TL&IS'] = home_system_planets['tidal_lock'] & home_system_planets['in_sphere']
tl_is = home_system_planets.groupby(['seed', 'star'])[['TL|IS', 'TL&IS', 'tidal_lock', 'in_sphere']].sum()
pd.DataFrame({col: tl_is.reset_index().groupby(col)['star'].count()
              for col in tl_is.columns}).fillna(0).astype(int)

,TL|IS,TL&IS,tidal_lock,in_sphere
0,2792,6721,6558,2868
1,4002,161,323,4014
2,88,0,1,0


time: 20.6 ms (started: 2021-10-13 12:07:47 +01:00)


In [14]:
res_df = star_df.copy()
res_df[vein_types + gas_types] = res_df[vein_types + gas_types] > 0
res_probs = res_df.groupby(['spectrum', 'type'])[ocean_types
                                                 + vein_types 
                                                 + gas_types
                                                ].mean()
res = 'Organic Crystal'
res_probs.loc[res_probs[res] > 0].sort_values(res, ascending=False).style.format(precision=2)

,,Water,Sulfuric acid,Iron,Copper,Silicon,Titanium,Stone,Coal,Oil,Fire ice,Kimberlite,Fractal Silicon,Organic Crystal,Optical Grating,Spiniform Stalagmite Crystal,Unipolar Magnets,Hydrogen gas,Deuterium gas,Fire ice gas
spectrum,type,,,,,,,,,,,,,,,,,,,
F,Main Sequence,0.79,0.39,1.00,1.00,1.00,1.00,1.00,1.00,0.54,0.68,0.31,0.21,0.44,0.19,0.26,0.00,0.46,0.25,0.22
G,Main Sequence,0.85,0.29,1.00,1.00,1.00,0.99,1.00,1.00,0.59,0.69,0.30,0.21,0.42,0.19,0.28,0.00,0.46,0.25,0.22
K,Main Sequence,0.85,0.18,0.99,0.99,0.98,0.96,0.99,0.99,0.50,0.78,0.31,0.27,0.41,0.23,0.31,0.00,0.31,0.17,0.15
M,Main Sequence,0.88,0.10,0.99,0.99,0.98,0.97,0.99,0.99,0.43,0.89,0.46,0.50,0.35,0.42,0.37,0.00,0.30,0.16,0.14
A,Main Sequence,0.54,0.53,1.00,1.00,1.00,1.00,1.00,1.00,0.29,0.56,0.51,0.33,0.23,0.30,0.17,0.00,0.53,0.29,0.26
B,Main Sequence,0.43,0.71,1.00,1.00,1.00,1.00,1.00,1.00,0.11,0.61,0.32,0.20,0.09,0.18,0.04,0.00,0.65,0.38,0.34
O,Main Sequence,0.38,0.83,1.00,1.00,1.00,1.00,1.00,1.00,0.11,0.70,0.79,0.59,0.08,0.57,0.08,0.00,0.68,0.41,0.36
M,Giant,0.01,0.37,1.00,1.00,0.94,0.94,1.00,1.00,0.01,0.00,0.46,0.29,0.01,0.25,0.01,0.00,0.54,0.29,0.25


time: 223 ms (started: 2021-10-13 12:07:47 +01:00)


In [15]:
def S_scaler(f):
    return '{:.2f}/s'.format(f)

def M_scaler(f):
    if f < 10e6:
        return '{:.1f}M'.format(f / 1e6)
    else:
        return '{:.0f}M'.format(f / 1e6)

formatters = {
    'limit': M_scaler,
    **{col: S_scaler for col in S_scaled},
    **{col: M_scaler for col in M_scaled},
    **{col + ' (Start)': S_scaler for col in S_scaled},
    **{col + ' (Start)': M_scaler for col in M_scaled},
}

def shader(series, pct):
    pct_subset = pct.loc[series.index, series.name] ** (1/3)
    cmap = mpl.cm.get_cmap('coolwarm_r')
    rgb = [sns.set_hls_values(color=cmap(x), h=None, l=0.75, s=None) for x in pct_subset]
    return ['background: {};'.format(mpl.colors.rgb2hex(x)) for x in rgb]

def show(full_df, df):
    shade_cols = [x for x in df.columns if x in formatters.keys()]
    pct = full_df[shade_cols].rank(pct=True, method='min')
    display(HTML(
        df.style
        .apply(func=shader, pct=pct, axis='index', subset=shade_cols)
        .format(formatter=formatters).to_html()
    ))

time: 736 µs (started: 2021-10-13 12:07:47 +01:00)


In [16]:
home_resources = home_systems.set_index('seed')[base_types + ['Oil', 'Fire ice', 'Fire ice gas']]
home_resources['limit'] =  home_resources[base_types].min(axis='columns')

seed_res = (
    tl_is.reset_index(level='star')
    .merge(
        seed_df,
        on='seed', validate='1:1')
    .merge(
        home_resources,
        on='seed', validate='1:1')
    .merge(
        star_df.groupby('seed')[ocean_types + vein_types + gas_types].sum(), 
        on='seed', validate='1:1', suffixes=[' (Start)', ''])
)
seed_res['ts'] = seed_res['ts'].dt.strftime('%b-%d')

time: 71.5 ms (started: 2021-10-13 12:07:47 +01:00)


In [17]:
enough_fire_ice = (
    (seed_res['Fire ice (Start)'] > 1e6) 
    | ((seed_res['Fire ice (Start)'] > 0) & (seed_res['Fire ice gas (Start)'] > 0))
)

potential = (
    seed_res
    .loc[enough_fire_ice]
    # .query('`TL|IS` >= 1')
    .query('`tidal_lock` >= 1')
    # .query('limit > 6e6')
    # .query('`Organic Crystal` > 50e6')
    
    .sort_values('limit', ascending=False)
    # .sort_values('Organic Crystal', ascending=False)
).set_index('seed')

pre_run = potential.index.isin(previous_runs)
show(potential, potential.loc[pre_run])
show(potential, potential.loc[~pre_run].head(50))


,star,TL|IS,TL&IS,tidal_lock,in_sphere,ts,Iron (Start),Copper (Start),Silicon (Start),Titanium (Start),Stone (Start),Coal (Start),Oil (Start),Fire ice (Start),Fire ice gas (Start),limit,Water,Sulfuric acid,Iron,Copper,Silicon,Titanium,Stone,Coal,Oil,Fire ice,Kimberlite,Fractal Silicon,Organic Crystal,Optical Grating,Spiniform Stalagmite Crystal,Unipolar Magnets,Hydrogen gas,Deuterium gas,Fire ice gas
seed,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
43378616,Alkaphrah,1,1,1,1,Sep-10,28M,21M,12M,18M,11M,9.3M,47.21/s,0.2M,0.62/s,9.3M,43,27,4725M,5994M,2824M,5600M,3998M,788M,2052.99/s,727M,132M,104M,71M,46M,54M,11M,24.42/s,0.72/s,16.47/s
26027001,Maasym,1,1,1,1,Sep-07,28M,23M,6.0M,27M,10M,9.2M,52.71/s,19M,0.65/s,6.0M,43,31,4672M,6191M,2957M,5649M,4318M,685M,2234.22/s,579M,83M,97M,58M,67M,67M,18M,15.96/s,0.33/s,19.52/s


,star,TL|IS,TL&IS,tidal_lock,in_sphere,ts,Iron (Start),Copper (Start),Silicon (Start),Titanium (Start),Stone (Start),Coal (Start),Oil (Start),Fire ice (Start),Fire ice gas (Start),limit,Water,Sulfuric acid,Iron,Copper,Silicon,Titanium,Stone,Coal,Oil,Fire ice,Kimberlite,Fractal Silicon,Organic Crystal,Optical Grating,Spiniform Stalagmite Crystal,Unipolar Magnets,Hydrogen gas,Deuterium gas,Fire ice gas
seed,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
28154786,GarnetStar,1,1,1,1,Oct-13,30M,22M,11M,14M,10.0M,11M,47.60/s,2.1M,0.62/s,10.0M,45,29,4617M,5991M,2348M,5454M,3741M,732M,2695.98/s,785M,74M,173M,78M,70M,98M,4.9M,23.35/s,1.72/s,8.79/s
95775847,Kerb,1,0,1,0,Oct-12,28M,22M,11M,17M,10M,9.8M,53.01/s,1.2M,0.00/s,9.8M,37,37,4702M,5942M,2591M,5368M,3939M,588M,1976.11/s,555M,149M,157M,52M,57M,33M,17M,26.75/s,1.95/s,8.86/s
16452863,Lambda Crateris,1,1,1,1,Sep-09,29M,23M,9.1M,15M,12M,9.8M,53.43/s,0.3M,0.58/s,9.1M,41,25,4078M,5103M,2167M,4715M,2741M,465M,2060.06/s,538M,184M,259M,48M,89M,82M,6.6M,26.11/s,0.82/s,14.93/s
37199889,ZenithStar,1,1,1,1,Oct-13,9.3M,7.1M,13M,16M,8.6M,10M,47.24/s,16M,0.00/s,7.1M,37,25,6029M,7935M,3177M,6719M,4744M,664M,2154.77/s,690M,178M,158M,74M,110M,116M,6.3M,31.93/s,2.34/s,14.34/s
84459702,Theemim,1,0,1,0,Sep-07,7.4M,7.0M,14M,17M,7.1M,11M,47.40/s,3.0M,0.65/s,7.0M,46,29,6018M,7125M,4117M,7553M,6425M,855M,2412.53/s,1388M,250M,373M,101M,161M,82M,20M,30.48/s,1.00/s,15.43/s
57418554,Haris,1,0,1,0,Sep-11,6.6M,11M,17M,16M,15M,9.9M,51.17/s,6.0M,0.00/s,6.6M,50,26,5703M,6938M,3314M,6841M,4253M,900M,2490.16/s,978M,94M,108M,82M,99M,55M,2.8M,19.93/s,0.61/s,12.95/s
91684768,Gamma Arae,1,1,1,1,Oct-13,30M,22M,6.3M,26M,8.4M,8.7M,47.68/s,1.5M,0.00/s,6.3M,40,27,4950M,6177M,2728M,5770M,4235M,664M,2219.23/s,655M,83M,97M,58M,62M,42M,14M,26.58/s,2.06/s,11.27/s
68258881,NairalZaurak,1,0,1,0,Sep-07,28M,21M,5.9M,27M,7.7M,8.2M,55.34/s,9.0M,0.00/s,5.9M,43,27,6214M,8254M,3508M,6947M,5554M,975M,2548.44/s,660M,65M,146M,108M,82M,66M,3.1M,22.02/s,0.62/s,16.87/s
98245197,Epsilon Pyxidis,1,1,1,1,Sep-06,28M,21M,5.5M,27M,7.0M,10M,49.04/s,1.5M,0.00/s,5.5M,45,28,4608M,5563M,2585M,5741M,4439M,580M,2039.22/s,921M,123M,113M,75M,77M,44M,6.9M,25.00/s,0.82/s,12.43/s


time: 117 ms (started: 2021-10-13 12:07:47 +01:00)
